## Before you start, upload the train.csv and test.csv files

In [ ]:
# Install necessary packages

# pip install seaborn

In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

import sklearn
from sklearn import preprocessing
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Ridge
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import PolynomialFeatures
from sklearn.linear_model import Lasso
from sklearn.metrics import mean_squared_error
from sklearn.linear_model import SGDRegressor
from sklearn.model_selection import KFold, cross_val_score
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import FunctionTransformer
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import GridSearchCV
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer

from scipy import stats
from math import ceil
from scipy.stats import probplot
from sklearn.model_selection import GridSearchCV, RepeatedKFold, cross_val_score
from sklearn.metrics import make_scorer
from sklearn.linear_model import LinearRegression, Lasso, Ridge, ElasticNet
from sklearn.svm import LinearSVR, SVR
from sklearn.neighbors import KNeighborsRegressor
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from xgboost import XGBRegressor
from sklearn.preprocessing import PolynomialFeatures

In [ ]:
init_df = pd.read_csv('train.csv')

In [ ]:
display(init_df)

# Show counts of NaN values in columns with NaN values
for i, col in zip(init_df.isna().sum(), init_df.columns):
  if i > 0:
      print('{:<20}{}'.format(col, i))

As seen in the dataframe there are several types of data, with a mix of categorical and numerical data. There are also multiple values missing, or listed as NaN. This means we are going to have to preprocess the data before we can conduct our analyses and models.

##Data Preprocessing and Exploratory Data Analysis

### Processing missing values

First split the dataset into a training and validation dataset *before* any of the missing values are processed.

We use an 80% training and 20% validation split. Test data is already in a separate file.

In [ ]:
# Validation set is a sample of 20%. Seed is used to get consistent results while testing
validate = init_df.sample(frac=0.2,random_state=200)
# training is set acquired by dropping the validation set
train = init_df.drop(validate.index)

# df is a copy of the training set, so we can keep training set unmodified
df = train.copy()


### Dividing the data into their respective levels of measurement

In [ ]:
# First divide data into the numerical or categorical, so we can preprocess it

# List of nominal and ordinal columns extracted from data_description.txt
nom_col = ['MSSubClass', 'MSZoning','Street','Alley','LandContour','LotConfig','Neighborhood','Condition1',
           'Condition2','BldgType','HouseStyle','RoofStyle','RoofMatl','Exterior1st','Exterior2nd',
           'BsmtFinType1','BsmtFinType2','Electrical','FireplaceQu','PavedDrive','Fence','SaleCondition',
           'MasVnrType','Foundation','Heating','CentralAir','GarageType','SaleType','MiscFeature']

ord_col = ['LotShape','Utilities','LandSlope','ExterQual','ExterCond','BsmtQual','BsmtCond','BsmtExposure',
           'HeatingQC','KitchenQual','Functional',
           'GarageFinish','GarageQual','GarageCond','PoolQC','OverallQual','OverallCond']

# List of all columns
allcat = list(df.columns[1:])

# List of numerical columns
num_col = [x for x in allcat if x not in nom_col+ord_col]


In [ ]:
# Check the discrete numerical features:
for i in df[num_col]:
    cats = list(set(list(df[i])))
    if len(cats) < 20:
        print("{:<20}{}".format(i, cats))
print()
pool = df[['PoolQC','PoolArea','SalePrice']].fillna(999)
for q, a, p in zip(pool.PoolQC, pool.PoolArea, pool.SalePrice):
    if q != 999:
        print(q, a, p)
print()
print(pool['PoolArea'].corr(pool['SalePrice']))

Clearly, features such as FullBath and such have very limited values. Hence, we will use these as ordinal data instead.
We also have numbers for the months and years sold, which can dictate the value, but is not related to how high the number is-- Hence, these should actually be considered nominal.

For PoolArea we also only have 6 houses in the dataset with pools, and we see that the correlation between PoolArea and SalePrice is 0.092, it is almost none. So, we will drop all pool-related columns from our dataset, since there are so few values, and, at least the area, has very little to do with the sale price.

So, in the df, we remove the Pool-related columns, we move MoSold and YrSold to the list of nominal columns, and we move everything else (FullBath, GarageCars, FirePlaces...) to the ordinal columns.
The remaining columns in the num_col will then actually be the numerical columns.

In [ ]:
remove = ['PoolQC' , 'PoolArea']
ordinals = []
nominals = ['MoSold' , 'YrSold']

# Check the discrete numerical features:
for i in df[num_col]:
    cats = list(set(list(df[i])))
    if len(cats) < 20:
        if i not in remove+nominals:
            ordinals.append(i)

# Fix the lists
num_col = [x for x in num_col if x not in remove+ordinals+nominals]
ord_col = ord_col+ordinals
ord_col.remove('PoolQC')
nom_col = nom_col+nominals

# Fix the df:
df.drop(remove, axis=1, inplace=True)

In [ ]:
#missing data
total = df.isnull().sum().sort_values(ascending=False)
percent = (df.isnull().sum()/df.isnull().count()).sort_values(ascending=False)*100
missing_data = pd.concat([total, percent], axis=1, keys=['Total', 'Percent'])
missing_data.head(20)

By looking into the data_description.txt, we see that a lot of the missing' NaN' values are not actually missing measurements, but rather meant to imply that the measured object was not present. For example, MiscFeatures is listed NaN for properties that did not have MiscFeatures

The solution to this is to find all categories that use NaN in this way, and substitute NaN for some other value-- For strings, this could be 'none', and for integers, this could be 0.

In [ ]:
# See the columns that contain NaN values:
for i in df.columns:
    if df[i].isna().sum() != 0:
        print('{:<20}{}'.format(i, df[i].isna().sum()))


Of these, drop electrical because there's only one property, drop MasVnrType because NaN means the value were not recorded. MasVnrArea NaNs are also dropped for the same reason. GarageYrBuilt is set to same year as the house (0 would be too far from all the other values), and the rest will have all 'NaN' replaced with 0 for ordinal columns and 'None' for nominal columns.

LotFrontage is the only continuous numerical value that is truly missing, based on our assumption that all properties should have a connected street. This will be estimated using multivariate imputation.

In [ ]:
# Drop the na rows from the df
df.dropna(subset = ['Electrical','MasVnrType','MasVnrArea'], inplace=True)

In [ ]:
def impute(input):

    #imp = IterativeImputer(max_iter=10, random_state=0)
    imp = IterativeImputer(n_nearest_features=None, imputation_order='ascending')
    imp.fit(input)
    output = pd.DataFrame(imp.transform(df), columns = df.columns)
    return output

def fixnan(input):
    # Fix the garage year built
    input.loc[input.GarageYrBlt.isnull(),'GarageYrBlt'] = input.loc[input.GarageYrBlt.isnull(),'YearBuilt']

    # Fix MasVnrArea as it is for some reason an object type
    input.MasVnrArea = input.MasVnrArea.astype(str).astype(float)

    # Fill NaNs
    for col in nom_col:
        input[col].fillna('None',inplace=True)
    for col in ord_col:
        input[col].fillna('None',inplace=True)

    # Remove LotFrontage since we don't want to fill this
    col_num_nan = num_col
    try: col_num_nan.remove('LotFrontage')
    except: pass

    for col in col_num_nan:
        input[col].fillna('0',inplace=True)
    return input

fixnan(df)

# Encode all nominal data as numerical with one-hot encoding to create regression
def onehot(data):
    output = pd.get_dummies(data=data, drop_first=True)
    output = pd.get_dummies(data=output, columns=['MSSubClass'])
    return output

# Encode ordinal data with arbitrary number sequences
def labenc(data):
    le = preprocessing.LabelEncoder()
    output = data.apply(preprocessing.LabelEncoder().fit_transform)
    return output

df[ord_col] = labenc(df[ord_col])

df = onehot(df)

#df = impute(df)

In [ ]:
"""

# Find categorical columns with intentional NaN values (see data_description.txt)
col_cat_nan = ['MiscFeature','Alley','Fence','MasVnrType','FireplaceQu',
           'GarageQual','GarageCond','GarageFinish','GarageType',
           'BsmtExposure','BsmtCond','BsmtQual','BsmtFinType1','BsmtFinType2']

# Find numerical columns with intentional NaN values (see data_description.txt)
col_num_nan = ['MasVnrArea','BsmtFullBath','BsmtHalfBath','BsmtFinSF1',
               'BsmtFinSF2','BsmtUnfSF','TotalBsmtSF','GarageArea','GarageCars']

# Use pandas built-in 'fillna' function to remove nan values:
def fillnan(input):
    for col in nom_col:
        input[col].fillna('None',inplace=True)

    for col in col_num_nan:
        input[col].fillna('0',inplace=True)
    return input

# Apply to df:
df = fillnan(df)

# Check which columns still contain NaN values:
#df.loc[:, df.isna().any()]

# There are 3. The first, GarageYrBlt, we will solve by just using the year
# that the house was built: YearBlt.
# The second, Electrical, only contains one single house where it was not
# measured. This can be checked with:
#   list(set(list(df['Electrical'])))
#   df['Electrical'].isna().value_counts()
# This is solved by dropping that single row from our dataset.
# The third is LotFrontage, a numerical measurement. There are too many here
# to simply drop the rows, so instead we will use the average of this value
# for these houses.

def multivar_impute(input):

  imp = IterativeImputer(max_iter=10, random_state=0)
  imp.fit(input)

  output = imp.transform(input)
  return output



# Perform on df:
df = fixnan(df)

# Perform on validate:
vali = fillnan(validate)
val = fixnan(vali)

"""

In [ ]:
"""
# Encode all nominal data as numerical with one-hot encoding to create regression
def onehot(data):
    output = pd.get_dummies(data=data, drop_first=True)
    output = pd.get_dummies(data=output, columns=['MSSubClass'])
    return output

# Apply to df:
df_nom = onehot(nom_col)

# Encode ordinal data with arbitrary number sequences
def labenc(data):
    le = preprocessing.LabelEncoder()
    output = data.apply(preprocessing.LabelEncoder().fit_transform)
    return output

# Apply to df:
df_ord = labenc(df[ord_col])
df = pd.concat([df_ord, df_nom, df_num], axis=1)



# Impute missing LotFrontage values with multivariate imputing
imputed = multivar_impute(df)
df = pd.DataFrame(imputed)
"""

In [ ]:
# Verify that there are no NaN values in df or val:
print(df.isnull().any().sum())
#print(val.isnull().any().sum())
for i in df.columns:
    if df[i].isnull().any().sum() == 1:
        print(i)
df

In [ ]:
skewlist = []
for i in df[num_col]:
    if abs(df[i].skew()) > 0.5:
        skewlist.append(i)
print(skewlist)

In [ ]:
# Plot the distributions of the skewed categories before and after using log transformation
# to see which ones work well for log transformation
for i, v in enumerate(df[skewlist]):
    print(v)
    gby = np.log1p(df[v])
    fig = plt.figure(figsize=(15,8))
    ax = fig.add_subplot(222)
    ax.set_title(v)
    res = probplot(gby, plot=ax)
    ax2 = fig.add_subplot(221)
    ax.set_title(v)
    res = probplot(df[v], plot=ax2)
    plt.show()
    print()

In [ ]:
logtransf = ['1stFlrSF','GrLivArea','SalePrice']

for i, v in enumerate(df[logtransf]):
  tmp = np.log(df[v])
  df[v] = tmp

### Plotting a correlation matrix of the numerical data

In [ ]:
# Plot numerical data in a correlation matrix
df_num = df[num_col]
# Create correlation between these columns of data
corr = df_num.corr()

plt.figure(figsize=(25,15))

# Create array of 0s in same shape as corr.shape
mask = np.zeros_like(corr)
# Take upper triangle of the shape of 0s
mask[np.triu_indices_from(mask)] = True

# Change mask and correlation to remove empty rows
mask = mask[1:, :-1]
corr = corr.iloc[1:,:-1].copy()

# Choose a diverging palette for the r values (0 should be white)
cmap = sns.diverging_palette(220, 20, as_cmap=True)

# Plot the heatmap
sns.heatmap(corr,         # Create heatmap of the df.corr created above
            mask=mask,    # Use the mask to not show repeated boxes
            annot=True,   # Annotate the squares with the r value
            fmt=".2f",    # Use 2 decimals
            annot_kws={"size": 8}, 
            cmap=cmap,    # cmap is the diverging one created above
            square=True,  # Square boxes look better
            cbar_kws={"shrink": .8}, # Shrink color bar to center it
            vmin=-1, vmax=1) # Max and Min values for the colors

plt.show()

It's pretty clear that there's some correlations, but most of the high correlations are pretty obvious or not very useful, for example 'GarageCars' and 'GarageArea' or 'YearBuilt' and 'GarageYrBlt'.

A few useful correlations are present, like GrLiveArea and SalePrice.### Plotting a correlation matrix of the ordinal data

### Plotting a correlation matrix of the ordinal data

In [ ]:
# Encode all ordinal data as numerical to create regression
df_ord = df[ord_col]
# Create correlation between these columns of data
ord_corr = df_ord.corr()

plt.figure(figsize=(25,15))

# Create array of 0s in same shape as corr.shape
mask = np.zeros_like(ord_corr)
# Take upper triangle of the shape of 0s
mask[np.triu_indices_from(mask)] = True

# Change mask and correlation to remove empty rows
mask = mask[1:, :-1]
ord_corr = ord_corr.iloc[1:,:-1].copy()

# Choose a diverging palette for the r values (0 should be white)
cmap = sns.diverging_palette(220, 20, as_cmap=True)

# Plot the heatmap
sns.heatmap(ord_corr,         # Create heatmap of the df.corr created above
            mask=mask,    # Use the mask to not show repeated boxes
            annot=True,   # Annotate the squares with the r value
            fmt=".2f",    # Use 2 decimals
            annot_kws={"size": 8}, 
            cmap=cmap,    # cmap is the diverging one created above
            square=True,  # Square boxes look better
            cbar_kws={"shrink": .8}, # Shrink color bar to center it
            vmin=-1, vmax=1) # Max and Min values for the colors

plt.show()

### Computing correlation coefficients for one-hot encoded nominal data

In [ ]:
# Encode all nominal data as numerical with one-hot encoding to create regression
#df[nom_col]['SalePrice'] = df['SalePrice']

corr_sp = df.corr()
print(corr_sp['SalePrice'].abs().sort_values(ascending=False)[1:])

topten = ['OverallQual','GrLivArea','GarageArea','1stFlrSF','FullBath','TotRmsAbvGrd','YearBuilt','Fireplaces','MasVnrArea','BsmtFinSF1']
topfive = ['OverallQual','GrLivArea','GarageCars','GarageArea','1stFlrSF']

In [ ]:
sns.set_context("paper", rc={"font.size":14,"axes.titlesize":14,"axes.labelsize":14})
# Encode all ordinal data as numerical to create regression
df_five = df[topfive]
# Create correlation between these columns of data
ord_corr = df_five.corr()

plt.figure(figsize=(3.1,3))

# Create array of 0s in same shape as corr.shape
mask = np.zeros_like(ord_corr)
# Take upper triangle of the shape of 0s
mask[np.triu_indices_from(mask)] = True

# Change mask and correlation to remove empty rows
mask = mask[1:, :-1]
ord_corr = ord_corr.iloc[1:,:-1].copy()

# Choose a diverging palette for the r values (0 should be white)
cmap = sns.diverging_palette(220, 20, as_cmap=True)

# Plot the heatmap
sns.heatmap(ord_corr,         # Create heatmap of the df.corr created above
            mask=mask,    # Use the mask to not show repeated boxes
            annot=True,   # Annotate the squares with the r value
            fmt=".2f",    # Use 2 decimals
            annot_kws={"size": 10}, 
            cmap=cmap,    # cmap is the diverging one created above
            square=True,  # Square boxes look better
            cbar_kws={"shrink": .8}, # Shrink color bar to center it
            vmin=-1, vmax=1).set_title("Five Features With Highest Correlation to Sale Price",fontsize=12) # Max and Min values for the colors

plt.show()

In [ ]:
# Encode all ordinal data as numerical to create regression
df_ten = df[topten]
# Create correlation between these columns of data
ord_corr = df_ten.corr()

plt.figure(figsize=(3.1,3))

# Create array of 0s in same shape as corr.shape
mask = np.zeros_like(ord_corr)
# Take upper triangle of the shape of 0s
mask[np.triu_indices_from(mask)] = True

# Change mask and correlation to remove empty rows
mask = mask[1:, :-1]
ord_corr = ord_corr.iloc[1:,:-1].copy()

# Choose a diverging palette for the r values (0 should be white)
cmap = sns.diverging_palette(220, 20, as_cmap=True)

# Plot the heatmap
sns.heatmap(ord_corr,         # Create heatmap of the df.corr created above
            mask=mask,    # Use the mask to not show repeated boxes
            annot=True,   # Annotate the squares with the r value
            fmt=".2f",    # Use 2 decimals
            annot_kws={"size": 8}, 
            cmap=cmap,    # cmap is the diverging one created above
            square=True,  # Square boxes look better
            cbar_kws={"shrink": .8}, # Shrink color bar to center it
            vmin=-1, vmax=1).set_title("Ten Features With Highest Correlation to Sale Price") # Max and Min values for the colors

plt.show()

## Exploring the target variable: SalePrice

### Plotting the distribution of the sale prices
First we plot a histogram and a normal qq-plot to asses the distribution

In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(16,5))
ax = sns.histplot(data=df.SalePrice, kde=True, ax=axes[0])
ax1 = stats.probplot(df.SalePrice.dropna(), plot=axes[1])
plt.show()

It is pretty clear from the above charts that SalePrice is *not* normally distributed with a clear right-skew.

This is not necessarily a problem in itself, but for some of our models this is not ideal, and a distribution that is closer to normal is preferred.
Fortunately, it does look like it conforms to a log-normal distribution, so there is a very simple method called log-transform that should transform it into something closer to a normal distribution.

In [ ]:
# Make a copy of dataframe
df_log = df.copy()
# The SalePrice column in the copy becomes the log of the SalePrice in df
df_log.SalePrice = np.log(df.SalePrice)

fig, axes = plt.subplots(1, 2, figsize=(16,5))
ax = sns.histplot(data=df_log.SalePrice, kde=True, ax=axes[0])
ax1 = stats.probplot(df_log.SalePrice.dropna(), plot=axes[1])
plt.show()

In [ ]:
# The new, 'standard' df will have:
# - one-hot encoded nominal categories
# - numerically encoded ordinal categories
# - All missing values filled in with the various methods performed so far
# - Log-transformed SalePrice to conform to normality
# df = pd.concat([df_log.SalePrice, df_ord.drop(['SalePrice'], axis=1), df_nom.drop(['SalePrice'], axis=1), df_num.drop(['SalePrice'], axis=1)], axis=1)

In [ ]:
# Which categories best predict the sale price?
corr_sp = df.corr()
print(corr_sp['SalePrice'].abs().sort_values(ascending=False)[1:])

## Linear Regression Model

### Using all processed data

In [ ]:
print(df[topten].isna().sum())
df = df.dropna(subset=topten)
#print(df.isna().sum())

In [ ]:
# Linear regression using all data

x_col = df.drop(['SalePrice'], axis=1)
X = x_col.values.reshape(-1, len(x_col.columns))
y = df['SalePrice'].values

reg = LinearRegression().fit(X, y)

print(reg.score(X, y))


Taking the top ten most correlated values gives reg.score of only 0.7910926699369514

In [ ]:
 def train_model(model, param_grid=[], X=[], y=[], 
                splits=5, repeats=5):

    # get unmodified training data, unless data to use already specified
    if len(y)==0:
        X,y = get_training_data()
    
    # create cross-validation method
    rkfold = RepeatedKFold(n_splits=splits, n_repeats=repeats)
    
    # perform a grid search if param_grid given
    if len(param_grid)>0:
        # setup grid search parameters
        gsearch = GridSearchCV(model, param_grid, cv=rkfold,
                               scoring=rmse_scorer,
                               verbose=1, return_train_score=True)

        # search the grid
        gsearch.fit(X,y)

        # extract best model from the grid
        model = gsearch.best_estimator_        
        best_idx = gsearch.best_index_

        # get cv-scores for best model
        grid_results = pd.DataFrame(gsearch.cv_results_)       
        cv_mean = abs(grid_results.loc[best_idx,'mean_test_score'])
        cv_std = grid_results.loc[best_idx,'std_test_score']

    # no grid search, just cross-val score for given model    
    else:
        grid_results = []
        cv_results = cross_val_score(model, X, y, scoring=rmse_scorer, cv=rkfold)
        cv_mean = abs(np.mean(cv_results))
        cv_std = np.std(cv_results)
    
    # combine mean and std cv-score in to a pandas series
    cv_score = pd.Series({'mean':cv_mean,'std':cv_std})

    # predict y using the fitted model
    y_pred = model.predict(X)
    
    # print stats on model performance         
    print('----------------------')
    print(model)
    print('----------------------')
    print('score=',model.score(X,y))
    print('rmse=',rmse(y, y_pred))
    print('cross_val: mean=',cv_mean,', std=',cv_std)



# places to store optimal models and scores
opt_models = dict()
score_models = pd.DataFrame(columns=['mean','std'])

# no. k-fold splits
splits=5
# no. k-fold iterations
repeats=5

def get_training_data():
    # extract training samples
    df_train = df
    
    # split SalePrice and features
    y = df_train.SalePrice
    X = df_train.drop('SalePrice',axis=1)
    
    return X, y

# metric for evaluation
def rmse(y_true, y_pred):
    diff = y_pred - y_true
    sum_sq = sum(diff**2)    
    n = len(y_pred)   
    
    return np.sqrt(sum_sq/n)

# function to detect outliers based on the predictions of a model
def find_outliers(model, X, y, sigma=3):

    # predict y values using model
    try:
        y_pred = pd.Series(model.predict(X), index=y.index)
    # if predicting fails, try fitting the model first
    except:
        model.fit(X,y)
        y_pred = pd.Series(model.predict(X), index=y.index)
        
    # calculate residuals between the model prediction and true y values
    resid = y - y_pred
    mean_resid = resid.mean()
    std_resid = resid.std()

    # calculate z statistic, define outliers to be where |z|>sigma
    z = (resid - mean_resid)/std_resid    
    outliers = z[abs(z)>sigma].index
    
    # print and plot the results
    print('R2=',model.score(X,y))
    print('rmse=',rmse(y, y_pred))
    print('---------------------------------------')

    print('mean of residuals:',mean_resid)
    print('std of residuals:',std_resid)
    print('---------------------------------------')

    print(len(outliers),'outliers:')
    print(outliers.tolist())

# scorer to be used in sklearn model fitting
rmse_scorer = make_scorer(rmse, greater_is_better=False)



In [ ]:
model = 'Ridge'

opt_models[model] = Ridge()
alph_range = np.arange(0.25,6,0.25)
param_grid = {'alpha': alph_range}

opt_models[model], cv_score, grid_results = train_model(opt_models[model], param_grid=param_grid)

cv_score.name = model
score_models = score_models.append(cv_score)

plt.figure()
plt.errorbar(alph_range, abs(grid_results['mean_test_score']),
             abs(grid_results['std_test_score'])/np.sqrt(splits*repeats))
plt.xlabel('alpha')
plt.ylabel('score');

In [ ]:
model ='ElasticNet'
opt_models[model] = ElasticNet()

param_grid = {'alpha': np.arange(1e-4,1e-3,1e-4),
              'l1_ratio': np.arange(0.1,1.0,0.1),
              'max_iter':[100000]}

opt_models[model], cv_score, grid_results = train_model(opt_models[model], param_grid=param_grid, splits=splits, repeats=repeats)

cv_score.name = model
score_models = score_models.append(cv_score)